In [9]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [8]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-jsv_ik29/gym-walk_7bcf52d4ab6348f496e915698ace67b1
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-jsv_ik29/gym-walk_7bcf52d4ab6348f496e915698ace67b1
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=4053 sha256=9d66aa02ee8e810e3abb318964858ec9c40425881dfb13d16d4bf339064e1686
  Stored in directory: /tmp/pip-ephem-wheel-cache-21kxs1sq/wheels/24/fe/c4/0cbc7511d29265bad7e28a09311db3f87f0cafba74af54d530
Successfully built gym-walk


In [10]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [11]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [39]:
custom = ["FHFF","SFFH","FFFH","HGFF"]
env = gym.make('FrozenLake-v1', desc = custom)
P = env.env.P
init_state = env.reset()
goal_state = 13
LEFT, RIGHT = range(2)

In [40]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, True)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, True),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, True),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(1.0, 1, 0, True)],
  1: [(1.0, 1, 0, True)],
  2: [(1.0, 1, 0, True)],
  3: [(1.0, 1, 0, True)]},
 2: {0: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, True),
   (0.3333333333333333, 6, 0.0, False)],
  1: [(0.3333333333333333, 1, 0.0, True),
   (0.3333333333333333, 6, 0.0, False),
   (0.3333333333333333, 3, 0.0, False)],
  2: [(0.3333333333333333, 6, 0.0, False),
   (0.3333333333333333, 3, 0.0, False),
   (0.3333333333333333, 2, 0.0, False)],
  3: [(0.3333333333333

Exponentially decaying schedule


In [18]:
def decay_schedule(
    init_value, min_value, decay_ratio,
    max_steps, log_start = -2, log_base=10):
    decay_steps = int(max_steps * decay_ratio)
    rem_steps = max_steps - decay_steps
    values = np.logspace(log_start, 0 , decay_steps,
                         base=log_base, endpoint=True)[::-1]
    values = (values - values.min()) / (values.max() - values.min())
    values = (init_value - min_value) * values + min_value
    values = np.pad(values, (0, rem_steps), 'edge')
    return values

Exploratory Policy Trajectories

In [20]:
from itertools import count
def generate_trajectory(
    select_action, Q, epsilon,
    env, max_steps=200):
    done, trajectory = False, []
    while not done:
        state = env.reset()
        for t in count():
            action = select_action(state, Q, epsilon)
            next_state, reward, done, _ = env.step(action)
            experience = (state, action, reward,
                          next_state, done)
            trajectory.append(experience)
            if done:
                break
            if t >= max_steps - 1:
                trajectory = []
                break
            state = next_state
        return np.array(trajectory)

Monte Carlo control

In [30]:
from tqdm import tqdm
# Pradeesh S 212221240038
def mc_control (env, gamma = 1.0,
                init_alpha = 0.5,min_alpha = 0.01, alpha_decay_ratio = 0.5,
                init_epsilon = 1.0, min_epsilon = 0.1, epsilon_decay_ratio = 0.9,
                n_episodes = 3000, max_steps = 200, first_visit = True):
  nS, nA = env.observation_space.n, env.action_space.n
  discounts = np.logspace(0, max_steps,num=max_steps,base=gamma,endpoint=False)
  alphas = decay_schedule(init_alpha,min_alpha,alpha_decay_ratio,n_episodes)
  epsilons = decay_schedule(init_epsilon,min_epsilon,epsilon_decay_ratio,n_episodes)
  pi_track = []
  Q = np.zeros((nS, nA))
  Q_track = np.zeros((n_episodes,nS,nA))
  select_action = lambda state, Q, epsilon: np.argmax(Q[state]) if np.random.random() > epsilon else np.random.randint(len(Q[state]))
  for e in tqdm(range(n_episodes), leave = False):
    trajectory = generate_trajectory(select_action,Q,epsilons[e],env,max_steps)
    visited = [[0]*nA for _ in range(nS)]

    for t, (state,action,reward,_,_) in enumerate(trajectory):
      state = int(state)
      action = int(action)
      #print(visited[state][action])
      if visited[state][action] and first_visit:
        continue
      visited[state][action] = 1
      n_steps = len(trajectory[t:])
      G = np.sum(discounts[:n_steps] * trajectory[t:,2])
      Q[state][action] = Q[state][action] + alphas[e] * (G - Q[state][action])
    Q_track[e] = Q
    pi_track.append(np.argmax(Q, axis=1))
  V = np.max(Q, axis=1)
  pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]

  return Q, V, pi, Q_track, pi_track

In [41]:
optimal_Q, optimal_V, optimal_pi,_,_ = mc_control(env)
print("Pradeesh S 212221240038")
print_state_value_function(optimal_Q, P, n_cols=4, prec=2, title='Action-value function:')
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='State-value function:')
print_policy(optimal_pi, P)

Pradeesh S 212221240038
Action-value function:
| 00 [0.53 0.24 0.28 0.26] |           | 02 [0.07 0.05 0.32 0.08] | 03 [0.14 0.03 0.01 0.03] |
| 04 [0.58 0.61 0.56 0.53] | 05 [0.3  0.58 0.36 0.3 ] | 06 [0.47 0.31 0.27 0.14] |           |
| 08 [0.35 0.37 0.41 0.64] | 09 [0.62 0.65 0.72 0.51] | 10 [0.6  0.38 0.29 0.3 ] |           |
|           |           | 14 [0.6  0.63 0.44 0.53] | 15 [0.39 0.36 0.2  0.25] |
State-value function:
| 00   0.53 |           | 02   0.32 | 03   0.14 |
| 04   0.61 | 05   0.58 | 06   0.47 |           |
| 08   0.64 | 09   0.72 | 10    0.6 |           |
|           |           | 14   0.63 | 15   0.39 |
Policy:
| 00      < |           | 02      > | 03      < |
| 04      v | 05      v | 06      < |           |
| 08      ^ | 09      > | 10      < |           |
|           |           | 14      v | 15      < |
